In [2]:
from webcrawler import (
    Crawler,
    Smartphone,
    Smartwatch,
    Laptop,
    Screen,
    Tablet,
    Earphone,
    Product,
)
import json, re

In [6]:
urls = [
    "https://www.thegioididong.com/dtdd/samsung-galaxy-s25-ultra",  # smartphone
    "https://www.thegioididong.com/dtdd/nokia-220-4g",  # phone
    "https://www.thegioididong.com/laptop/macbook-air-13-inch-m4-16gb-256gb-sac-70w",  # mac
    "https://www.thegioididong.com/laptop/lenovo-thinkpad-v14-i7-83a00012vn?code=0220042002725",  # laptop
    "https://www.thegioididong.com/tai-nghe/tai-nghe-tws-jbl-wave-beam-2?utm_flashsale=1",  # tws
    "https://www.thegioididong.com/tai-nghe/tai-nghe-co-day-gaming-hp-hyperx-cloud-stinger-core-ii",  # headphone
    "https://www.thegioididong.com/tai-nghe/tai-nghe-ep-oppo-mh135",  # jack
    "https://www.thegioididong.com/man-hinh-may-tinh/samsung-s3-s30gd-ls27d300gaexxv-27-inch-fhd-ips-100hz-5ms",  # screen
    "https://www.thegioididong.com/may-tinh-bang/xiaomi-redmi-pad-se-8gb?utm_flashsale=1",  # tablet
    "https://www.thegioididong.com/dong-ho-thong-minh/xiaomi-watch-s-3?utm_flashsale=1",  # watch
    "https://www.thegioididong.com/dong-ho-thong-minh/mi-band-9-active?utm_flashsale=1",  # band
]

d = await Crawler.async_inspect(
    urls[7], xpath="//*[@id='productld']/text()", encoding="utf-8"
)


d = json.loads(d[0])
d

/tmp/ipykernel_11011/293122836.py:304: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  }, orient="index").transpose().fillna(value=np.nan)


{'@context': 'https://schema.org',
 '@type': 'Product',
 'name': 'Màn hình Samsung S3 S30GD LS27D300GAEXXV 27 inch FHD/IPS/100Hz/5ms',
 'url': 'https://www.thegioididong.com/man-hinh-may-tinh/samsung-s3-s30gd-ls27d300gaexxv-27-inch-fhd-ips-100hz-5ms',
 'image': {'@type': 'ImageObject',
  'contentUrl': 'https://cdnv2.tgdd.vn/mwg-static/tgdd/Products/Images/5697/339897/samsung-s3-s30gd-ls27d300gaexxv-27-inch-fhd-ips-100hz-5ms-den-thumb-638867187940665986-600x600.jpg'},
 'description': 'Samsung S3 S30GD 27 inch Full HD LS27D300GAEXXV chính hãng - Mua online, hỗ trợ trả góp xét duyệt nhanh. Thanh toán tại nhà, giao hàng miễn phí trong 1 giờ. Bảo hành trên toàn quốc. Xem ngay!',
 'sku': '339897',
 'mpn': '339897',
 'brand': {'@type': 'Brand', 'name': ['Samsung']},
 'review': None,
 'aggregateRating': {'@type': 'AggregateRating',
  'bestRating': 5,
  'worstRating': 1,
  'ratingValue': 5.0,
  'reviewcount': 80,
  'description': None},
 'additionalProperty': [{'@type': 'PropertyValue',
   'nam

In [7]:
plist = []


def parse_common_info(data: dict):
    prd = Product(
        product_id=data["sku"].strip(),
        name=data["name"].strip(),
        price=int(data["offers"]["price"]),
        brand=data["brand"]["name"][0].strip(),
        url=data["url"].strip(),
    )

    if data["aggregateRating"]:
        prd.rating = data["aggregateRating"]["ratingValue"]
        prd.reviews_count = int(data["aggregateRating"]["reviewcount"])

    released_value = [
        i["value"].strip()
        for i in data["additionalProperty"]
        if i["name"] == "Thời điểm ra mắt"
        or i["name"] == "Thời gian ra mắt"
        or i["name"] == "Năm ra mắt"
    ]
    prd.release_date = released_value[0] if released_value else None

    # check device type
    weight_value = [
        i["value"].split(" - ")[-1].strip()
        for i in data["additionalProperty"]
        if i["name"] == "Kích thước, khối lượng" or i["name"] == "Khối lượng"
    ]
    jack_value = [
        i["value"].strip()
        for i in data["additionalProperty"]
        if i["name"] == "Jack cắm"
    ]

    if prd.url.split("/")[3] == "laptop":  # classify by url hint
        prd.category = "Laptop"
    elif prd.url.split("/")[3] == "may-tinh-bang":
        prd.category = "Tablet"
    elif prd.url.split("/")[3] == "man-hinh-may-tinh":
        prd.category = "Screen"
    else:  # classify by weight
        if weight_value:
            g_vals = re.findall(r"(\d+\.?\d*)\s?(?:g|\()", weight_value[0])
            gam = float(g_vals[0]) if g_vals else None  # actual weight value in gam

            if prd.url.split("/")[3] == "dtdd":
                prd.category = "Smartphone" if gam and gam > 135.0 else "Phone"
            elif prd.url.split("/")[3] == "dong-ho-thong-minh":
                prd.category = "Smartwatch" if gam and gam > 30.0 else "Smartband"
            else:
                prd.category = (
                    "Headphone"
                    if gam and gam > 100.0
                    else "Earphone" if jack_value else "Earbuds"
                )

    return {
        "product_id": prd.product_id,
        "name": prd.name,
        "price": prd.price,
        "brand": prd.brand,
        "category": prd.category,
        "rating": prd.rating,
        "reviews_count": prd.reviews_count,
        "url": prd.url,
        "release_date": prd.release_date,
    }


def parse_specs_info(data: dict, type: str):
    specs = {
        "cpu": None,
        "cpu_speed": None,
        "gpu": None,
        "ram": None,
        "ram_tech": None,
        "storage": None,
        "chipset": None,
        "chipset_speed": None,
        "scr_size": None,
        "scr_tech": None,
        "scr_res": None,
        "scr_freq": None,
        "os": None,
        "battery": None,
        "case_battery": None,
        "connectivity": None,
        "control": None,
        "power_csp": None,
        "ports": None,
    }

    if type == "smartphone":
        chip_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Chip xử lý (CPU)"
        ]
        specs["chipset"] = chip_value[0] if chip_value else None

        ram_value = [
            i["value"].strip() for i in data["additionalProperty"] if i["name"] == "RAM"
        ]
        specs["ram"] = ram_value[0] if ram_value else None

        disk_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Dung lượng lưu trữ"
        ]
        specs["storage"] = disk_value[0] if disk_value else None

        scsize_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Màn hình rộng"
        ]
        specs["scr_size"] = (
            re.findall(r'\d\.?\d*"', scsize_value[0])[0]
            if scsize_value and re.findall(r'\d\.?\d*"', scsize_value[0])
            else None
        )
        specs["scr_freq"] = (
            re.findall(r">\s*(\d+\s*Hz)\s*<", scsize_value[0])[0]
            if scsize_value and re.findall(r">\s*(\d+\s*Hz)\s*<", scsize_value[0])
            else None
        )

        sctech_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Công nghệ màn hình"
        ]
        specs["scr_tech"] = sctech_value[0] if sctech_value else None

        scres_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Độ phân giải màn hình"
        ]
        specs["scr_res"] = scres_value[0] if scres_value else None

        os_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Hệ điều hành"
        ]
        specs["os"] = os_value[0] if os_value else None

        battery_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Dung lượng pin"
        ]
        specs["battery"] = battery_value[0] if battery_value else None

        return {
            "chipset": specs["chipset"],
            "ram": specs["ram"],
            "storage": specs["storage"],
            "scr_size": specs["scr_size"],
            "scr_tech": specs["scr_tech"],
            "scr_res": specs["scr_res"],
            "scr_freq": specs["scr_freq"],
            "os": specs["os"],
            "battery": specs["battery"],
        }
    elif type == "laptop":
        cpu_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Công nghệ CPU"
        ]
        specs["cpu"] = cpu_value[0] if cpu_value else None

        cpuspd_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Tốc độ CPU"
        ]
        specs["cpu_speed"] = cpuspd_value[0] if cpuspd_value else None

        gpu_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Card màn hình"
        ]
        specs["gpu"] = (
            " ".join(
                [
                    i.removesuffix("</a>").split(">")[-1]
                    for i in gpu_value[0].split(" - ")
                ]
            )
            if gpu_value
            else None
        )

        ram_value = [
            i["value"].strip() for i in data["additionalProperty"] if i["name"] == "RAM"
        ]
        specs["ram"] = ram_value[0] if ram_value else None

        ramtech_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Loại RAM" or i["name"] == "Tốc độ Bus RAM"
        ]
        specs["ram_tech"] = " ".join(ramtech_value) if ramtech_value else None

        disk_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Ổ cứng"
        ]
        specs["storage"] = disk_value[0] if disk_value else None

        scsize_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Kích thước màn hình"
        ]
        specs["scr_size"] = scsize_value[0] if scsize_value else None

        scfreq_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Tần số quét"
        ]
        specs["scr_freq"] = scfreq_value[0] if scfreq_value else None

        sctech_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Tấm nền"
        ]
        specs["scr_tech"] = sctech_value[0] if sctech_value else None

        scres_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Độ phân giải"
        ]
        specs["scr_res"] = scres_value[0] if scres_value else None

        os_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Hệ điều hành"
        ]
        specs["os"] = os_value[0] if os_value else None

        battery_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Thông tin Pin"
        ]
        specs["battery"] = battery_value[0] if battery_value else None

        return {
            "cpu": specs["cpu"],
            "cpu_speed": specs["cpu_speed"],
            "gpu": specs["gpu"],
            "ram": specs["ram"],
            "ram_tech": specs["ram_tech"],
            "storage": specs["storage"],
            "scr_size": specs["scr_size"],
            "scr_tech": specs["scr_tech"],
            "scr_res": specs["scr_res"],
            "scr_freq": specs["scr_freq"],
            "os": specs["os"],
            "battery": specs["battery"],
        }
    elif type == "tablet":
        chip_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Chip xử lý (CPU)"
        ]
        specs["chipset"] = chip_value[0] if chip_value else None

        chipspd_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Tốc độ CPU"
        ]
        specs["chipset_speed"] = chipspd_value[0] if chipspd_value else None

        gpu_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Chip đồ hoạ (GPU)"
        ]
        specs["gpu"] = gpu_value[0] if gpu_value else None

        ram_value = [
            i["value"].strip() for i in data["additionalProperty"] if i["name"] == "RAM"
        ]
        specs["ram"] = ram_value[0] if ram_value else None

        disk_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Dung lượng lưu trữ"
        ]
        specs["storage"] = disk_value[0] if disk_value else None

        scsize_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Màn hình rộng"
        ]
        specs["scr_size"] = (
            re.findall(r'\d\.?\d*"', scsize_value[0])[0]
            if scsize_value and re.findall(r'\d\.?\d*"', scsize_value[0])
            else None
        )
        specs["scr_freq"] = (
            re.findall(r">\s*(\d+\s*Hz)\s*<", scsize_value[0])[0]
            if scsize_value and re.findall(r">\s*(\d+\s*Hz)\s*<", scsize_value[0])
            else None
        )

        sctech_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Công nghệ màn hình"
        ]
        specs["scr_tech"] = sctech_value[0] if sctech_value else None

        scres_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Độ phân giải"
        ]
        specs["scr_res"] = scres_value[0] if scres_value else None

        os_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Hệ điều hành"
        ]
        specs["os"] = os_value[0] if os_value else None

        battery_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Dung lượng pin"
        ]
        specs["battery"] = battery_value[0] if battery_value else None

        return {
            "chipset": specs["chipset"],
            "chipset_speed": specs["chipset_speed"],
            "gpu": specs["gpu"],
            "ram": specs["ram"],
            "storage": specs["storage"],
            "scr_size": specs["scr_size"],
            "scr_tech": specs["scr_tech"],
            "scr_res": specs["scr_res"],
            "scr_freq": specs["scr_freq"],
            "os": specs["os"],
            "battery": specs["battery"],
        }
    elif type == "smartwatch":
        chip_value = [
            i["value"].strip() for i in data["additionalProperty"] if i["name"] == "CPU"
        ]
        specs["chipset"] = chip_value[0] if chip_value else None

        disk_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Bộ nhớ trong"
        ]
        specs["storage"] = disk_value[0] if disk_value else None

        scsize_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Kích thước màn hình"
        ]
        specs["scr_size"] = scsize_value[0] if scsize_value else None

        sctech_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Công nghệ màn hình"
        ]
        specs["scr_tech"] = sctech_value[0] if sctech_value else None

        os_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Hệ điều hành"
        ]
        specs["os"] = os_value[0] if os_value else None

        cnt_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Kết nối"
        ]
        specs["connectivity"] = cnt_value[0] if cnt_value else None

        battery_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Dung lượng pin"
        ]
        specs["battery"] = battery_value[0] if battery_value else None

        return {
            "chipset": specs["chipset"],
            "storage": specs["storage"],
            "scr_size": specs["scr_size"],
            "scr_tech": specs["scr_tech"],
            "os": specs["os"],
            "connectivity": specs["connectivity"],
            "battery": specs["battery"],
        }
    elif type == "earphone":
        ctr_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Điều khiển"
        ]
        specs["control"] = ctr_value[0] if ctr_value else None

        battery_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Thời lượng pin tai nghe"
        ]
        specs["battery"] = battery_value[0] if battery_value else None

        casebtr_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Thời lượng pin hộp sạc"
        ]
        specs["case_battery"] = casebtr_value[0] if casebtr_value else None

        cnt_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Công nghệ kết nối"
        ]
        specs["connectivity"] = cnt_value[0] if cnt_value else None

        return {
            "connectivity": specs["connectivity"],
            "battery": specs["battery"],
            "case_battery": specs["case_battery"],
            "control": specs["control"],
        }
    else:
        scsize_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Kích thước màn hình"
        ]
        specs["scr_size"] = scsize_value[0] if scsize_value else None

        scfreq_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Tần số quét"
        ]
        specs["scr_freq"] = scfreq_value[0] if scfreq_value else None

        sctech_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Tấm nền"
        ]
        specs["scr_tech"] = sctech_value[0] if sctech_value else None

        scres_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Độ phân giải"
        ]
        specs["scr_res"] = scres_value[0] if scres_value else None

        ports_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Cổng kết nối"
        ]
        specs["ports"] = ports_value[0] if ports_value else None

        pwcsp_value = [
            i["value"].strip()
            for i in data["additionalProperty"]
            if i["name"] == "Công suất tiêu thụ điện"
        ]
        specs["power_csp"] = pwcsp_value[0] if pwcsp_value else None

        return {
            "scr_size": specs["scr_size"],
            "scr_tech": specs["scr_tech"],
            "scr_res": specs["scr_res"],
            "scr_freq": specs["scr_freq"],
            "ports": specs["ports"],
            "power_csp": specs["power_csp"],
        }


for i in urls:
    product = None
    data = await Crawler.async_inspect(
        i, xpath="//*[@id='productld']/text()", encoding="utf-8"
    )

    data = json.loads(data[0])

    # check device type
    if i.split("/")[3] == "dtdd":
        product = Smartphone(
            **parse_common_info(data), **parse_specs_info(data, "smartphone")
        )
    elif i.split("/")[3] == "laptop":
        product = Laptop(**parse_common_info(data), **parse_specs_info(data, "laptop"))
    elif i.split("/")[3] == "may-tinh-bang":
        product = Tablet(**parse_common_info(data), **parse_specs_info(data, "tablet"))
    elif i.split("/")[3] == "dong-ho-thong-minh":
        product = Smartwatch(
            **parse_common_info(data), **parse_specs_info(data, "smartwatch")
        )
    elif i.split("/")[3] == "tai-nghe":
        product = Earphone(
            **parse_common_info(data), **parse_specs_info(data, "earphone")
        )
    else:
        product = Screen(**parse_common_info(data), **parse_specs_info(data, "screen"))

    # parse common info

    plist.append(product.info())


plist

[{'id': '333347',
  'name': 'Điện thoại Samsung Galaxy S25 Ultra 5G 12GB/256GB',
  'price': 26880000,
  'brand': 'Samsung',
  'category': 'Smartphone',
  'chipset': 'Qualcomm Snapdragon 8 Elite For Galaxy 8 nhân',
  'ram': '12 GB',
  'storage': '256 GB',
  'screen_size': '6.9"',
  'screen_tech': 'Dynamic AMOLED 2X',
  'screen_resolution': '2K+ (1440 x 3120 Pixels)',
  'screen_frequency': '120 Hz',
  'operating_system': 'Android 15',
  'battery': '5000 mAh',
  'rating': 4.9,
  'reviews_count': 13380,
  'url': 'https://www.thegioididong.com/dtdd/samsung-galaxy-s25-ultra',
  'created_at': '01/2025',
  'updated_at': '2025-09-01 01:39:42'},
 {'id': '207956',
  'name': 'Điện thoại Nokia 220 4G',
  'price': 980000,
  'brand': 'Nokia',
  'category': 'Phone',
  'chipset': 'Unisoc T107 8 nhân',
  'ram': '64 MB',
  'storage': '128 MB',
  'screen_size': '2.8"',
  'screen_tech': None,
  'screen_resolution': 'QVGA (240 x 320 Pixels)',
  'screen_frequency': None,
  'operating_system': 'S30+',
  'batt